In [57]:
import pandas as pd
import glob
from sklearn.feature_extraction.text import CountVectorizer

In [39]:
glob.glob('bias_data/WNC/*')

['bias_data/WNC/biased.word.dev',
 'bias_data/WNC/revision_topics.csv',
 'bias_data/WNC/biased.word.test',
 'bias_data/WNC/neutral',
 'bias_data/WNC/biased.word.train',
 'bias_data/WNC/biased.full']

## Reading in data

Let's start out with the train / dev subsets proposed by the authors. 

In [77]:
train = pd.read_csv('bias_data/WNC/biased.word.train', sep='\t', header=None)
dev = pd.read_csv('bias_data/WNC/biased.word.dev', sep='\t', header=None)
test = pd.read_csv('bias_data/WNC/biased.word.test', sep='\t', header=None)

In [78]:
print("Train length: {}".format(len(train)))
print("Dev length: {}".format(len(dev)))
print("Test length: {}".format(len(test)))

Train length: 53803
Dev length: 700
Test length: 1000


In [101]:
def read_process_data(train_path, dev_path):
    train = pd.read_csv(train_path, sep='\t', header=None)
    dev = pd.read_csv(dev_path, sep = '\t', header=None)
    
    print("Train length: {}".format(len(train)))
    print("Dev length: {}".format(len(dev)))
    
    #Rename columns
    train.columns = ['id', 'annot_old', 'annot_new', 'biased', 'unbiased', 'tags', 'roots']
    dev.columns = ['id', 'annot_old', 'annot_new', 'biased', 'unbiased', 'tags', 'roots']
    
    #Process and recombine training data: 
    train_biased, dev_biased = pd.DataFrame(train.biased), pd.DataFrame(dev.biased)
    train_unbiased, dev_unbiased = pd.DataFrame(train.unbiased), pd.DataFrame(dev.unbiased)
    
    
    train_biased['label'], dev_biased['label'] = [1]*len(train_biased), [1]*len(dev_biased)
    train_unbiased['label'], dev_unbiased['label'] = [0]*len(train_unbiased), [0]*len(dev_unbiased)

    #Fix colnames
    train_biased.columns, dev_biased.columns = ['text', 'label'], ['text', 'label']
    train_unbiased.columns, dev_unbiased.columns = ['text', 'label'], ['text', 'label']
    #Combine
    train_all, dev_all = pd.concat([train_biased, train_unbiased]), pd.concat([dev_biased, dev_unbiased])
    
    return train_all, dev_all

In [102]:
train, dev = read_process_data('bias_data/WNC/biased.word.train', 'bias_data/WNC/biased.word.dev')

Train length: 53803
Dev length: 700


In [79]:
train.columns = ['id', 'annot_old', 'annot_new', 'biased', 'unbiased', 'tags', 'roots']
dev.columns = ['id', 'annot_old', 'annot_new', 'biased', 'unbiased', 'tags', 'roots']

In [80]:
train_biased = pd.DataFrame(train.biased)
train_unbiased = pd.DataFrame(train.unbiased)

#Add labels
train_biased['label'] = [1]*len(train_biased)
train_unbiased['label'] = [0]*len(train_unbiased)

#Fix colnames
train_biased.columns = ['text', 'label']
train_unbiased.columns = ['text', 'label']

In [81]:
#Shmush
train_all = pd.concat([train_biased, train_unbiased])

In [104]:
dev

,text,label
0,in addition to sponsoring palestinian terror a...,1
1,the game is currently played in 47 countries w...,1
2,no part of the valley lies in the area current...,1
3,scholars perceived that it was discordant with...,1
4,"since the chinese civil war in 1949, taiwan ha...",1
...,...,...
695,in 2008 five pharmaceutical companies received...,0
696,"the palm, a steakhouse restaurant chain origin...",0
697,d.c. united's early successes,0
698,on 29 june 2007 price gave birth to her third ...,0


In [ ]:
dev_biased = pd.DataFrame(dev.biased)

## Construct bag-of-words representations.

In [105]:
#Initialize vectorizer
vectorizer = CountVectorizer()

In [106]:
X_train = vectorizer.fit_transform(train.text)
y_train = train.label

X_dev = vectorizer.transform(dev.text)
y_dev = dev.label

In [109]:
print(X_train.shape)
print(X_dev.shape)


(107606, 79211)
(1400, 79211)


In [130]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

### Attempt 1A: Plain bag of words into logistic regression

In [116]:
#Was running out of iterations hahaa
lr = LogisticRegression(max_iter=1000)

In [117]:
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [126]:
lr.predict(X_train)

array([1, 1, 1, ..., 0, 1, 0])

In [129]:
print("Training set accuracy: {}".format(lr.score(X_train, y_train)))
print("Val set accuracy: {}".format(lr.score(X_dev, y_dev)))

Training set accuracy: 0.77603479359887
Val set accuracy: 0.6814285714285714


### Attempt 1B: Plain bag of words into random forest
Notice the SEVERE overfitting here, which is strange given the model used. 

In [131]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [132]:
print("Training set accuracy: {}".format(rf.score(X_train, y_train)))
print("Val set accuracy: {}".format(rf.score(X_dev, y_dev)))

Training set accuracy: 0.9942103600170994
Val set accuracy: 0.6757142857142857


### Attempt 2: TF-IDF

In [134]:
from sklearn.feature_extraction.text import TfidfVectorizer as Tfidf

In [135]:
tfidf = Tfidf()
X_train = tfidf.fit_transform(train.text)
y_train = train.label

X_dev = tfidf.transform(dev.text)
y_dev = dev.label

### Attempt 2a: TF-IDF Logistic Regression

In [136]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
#This one finishes in way fewer iterations! Interesting

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [138]:
print("Training set accuracy: {}".format(lr.score(X_train, y_train)))
print("Val set accuracy: {}".format(lr.score(X_dev, y_dev)))

Training set accuracy: 0.706196680482501
Val set accuracy: 0.6842857142857143


### Attempt 2b: TF-IDF Random Forest
Still overfitting, which makes sense...

In [139]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [140]:
print("Training set accuracy: {}".format(rf.score(X_train, y_train)))
print("Val set accuracy: {}".format(rf.score(X_dev, y_dev)))

Training set accuracy: 0.9942475326654647
Val set accuracy: 0.6564285714285715


In [142]:
pd.Series(y_dev).value_counts()

1    700
0    700
Name: label, dtype: int64